# Unitary transformations of boson operators

In [2]:
from sympy import *
init_printing()

from sympy.physics.quantum import *
from sympy.physics.quantum.boson import *
from sympy.physics.quantum.fermion import *
from sympy.physics.quantum.operatorordering import *

In [3]:
from sympy_quantum_utils import *

/home/chris/Dokumente/git/sympsi-notebooks/sympy_quantum_utils.py:13: UserWarning: Deprecated: Use sympy_utils_extra instead
  warnings.warn("Deprecated: Use sympy_utils_extra instead")


In [4]:
omega_0, t, alpha = symbols("omega_0, t, alpha")
n = 8

In [5]:
a = BosonOp("a")

## Transformations with $U = \exp(i \omega_0 a^\dagger a t)$

In [6]:
A = I * omega_0 * Dagger(a) * a * t
U = exp(A)

U

### $U a^\dagger a U^\dagger = e^{i \omega_0 a^\dagger a t} a^\dagger a e^{-i \omega_0 a^\dagger a t} = a^\dagger a$

In [7]:
e = bch_expansion(A, Dagger(a) * a, n).doit()

normal_ordered_form(e)

In [8]:
unitary_transformation(U, Dagger(a) * a)

### $U a U^\dagger = a e^{-i\omega_0t}$

In [9]:
unitary_transformation(U, a)

In [11]:
e = bch_expansion(A, a, n).doit()

e = normal_ordered_form(expand(e))

e = collect(e, a)

e

In [12]:
e == (exp(-I * omega_0 * t).series(t, 0, n=n).removeO()) * a

False

### $U a^\dagger U^\dagger = a^\dagger e^{i\omega_0t}$

In [14]:
unitary_transformation(U, Dagger(a))

In [15]:
e = bch_expansion(A, Dagger(a), n).doit()

e = normal_ordered_form(expand(e))

e = collect(e, Dagger(a))

e

In [16]:
e == (exp(I * omega_0 * t).series(t, 0, n=n).removeO()) * Dagger(a)

False

## Transformations with $U = D(\alpha) = e^{\alpha a^\dagger - \alpha^* a}$:

In [21]:
H = Dagger(a) * alpha - conjugate(alpha) * a
U = exp(H)

U

### $U a U^\dagger = a - \alpha$

In [22]:
unitary_transformation(U, a)

### $U a^\dagger U^\dagger = a^\dagger - \alpha^*$

In [23]:
unitary_transformation(U, Dagger(a))

### $U a^\dagger a U^\dagger = a^\dagger a - \alpha a^\dagger - \alpha^* a + |\alpha|^2$

In [24]:
unitary_transformation(U, Dagger(a) * a)

## Optomechanical system

Consider the standard Hamiltonian for an optomechanical syste, including a classical driving signal on the optical mode:

$$
H = \hbar\omega_a a^\dagger a + \hbar \omega_b b^\dagger b + \hbar g a^\dagger a (b + b^\dagger) + (A e^{-i\omega_d t} + A^* e^{i\omega_d t})(a + a^\dagger)
$$



In [25]:
n = 6
omega_a, omega_b, g, A, Delta = symbols("omega_a, omega_b, g, A, Delta", positive=True)
Hsym, omega_d = symbols("H, omega_d")

In [26]:
a, b = BosonOp("a"), BosonOp("b")

In [27]:
H = omega_a * Dagger(a) * a + omega_b * Dagger(b) * b + g * Dagger(a) * a * (b + Dagger(b)) \
    + (A * exp(-I * omega_d * t) + conjugate(A) * exp(I * omega_d * t)) * (a + Dagger(a))

Eq(Hsym, H)

### Linearized interaction

First we apply the unitary transformation $U = e^{i \omega_d a^\dagger a t}$:

In [28]:
U = exp(I * Dagger(a) * a * omega_d * t)

U

In [29]:
unitary_transformation(U, H, independent=True)

In [30]:
H1 = hamiltonian_transformation(U, H, independent=True)

H1

We can now perform a rotating-wave approximation (RWA) by eliminating all terms that rotate with frequencies $2\omega_d$:

In [31]:
H2 = drop_terms_containing(H1.expand(), [exp(-2*I*omega_d*t), exp(2*I*omega_d*t)])

Eq(Symbol("H_{rwa}"), H2)

Now we introduce the detuning $\Delta = \omega_a - \omega_d$:

In [32]:
H3 = H2.subs(omega_a, Delta + omega_d).expand()

H3

To eliminate the coherent part of the state of the cavity mode we apply the unitary displacement operator $U = e^{\alpha a^\dagger - \alpha^*a}$:

In [33]:
alpha = symbols("alpha")

In [34]:
H = Dagger(a) * alpha - conjugate(alpha) * a
U = exp(H)

U

In [35]:
H4 = hamiltonian_transformation(U, H3, independent=True)

H4

Now want to cancel out the drivng terms so we set $A - \Delta \alpha = 0$, i.e. $\alpha = A/\Delta$:

In [36]:
H5 = H4.expand().subs({A: alpha * Delta, conjugate(alpha): alpha}) 

H5 = collect(H5, [g * Dagger(a) * a, - alpha * g])

H5

Drop C-numbers from the hamiltonian:

In [37]:
H6 = drop_c_number_terms(H5)

H6

Now, if driving strength is large, so that $\alpha \gg 1$, we can drop the nonlinear interaction term, and we have an linear effective coupling:

In [38]:
H7 = drop_terms_containing(H6, [g * Dagger(a) * a])  # XXX not working

Eq(Hsym, H7)

## Versions

In [39]:
%reload_ext version_information

%version_information sympy

ModuleNotFoundError: No module named 'version_information'